<a href="https://colab.research.google.com/github/tiagopatriciosantos/Hashnode_Notebooks/blob/master/BTC_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time, datetime
import os
import pandas as pd

def convert_date_to_UTC_timestamp(strdate):
  '''
  Converts string into UTC timestamp in milliseconds, integer
  params:
    strdate - with format YYYYmmDD or YYYYmmDDHHMMSS  ex: 20170101 or 20170101000001
  returns:
    int - UTC timestamp in milliseconds
  ''' 
  strdate = strdate.ljust(14,"0")
  date_obj = datetime.datetime.strptime(strdate, '%Y%m%d%H%M%S')
  return int( time.mktime(date_obj.timetuple()) * 1000 )

def convert_UTC_timestamp_to_date_obj(UTC_timestamp):
  '''
  Converts UTC timestamp in milliseconds, integer, into string date fromated '%Y-%m-%d %H:%M:%S'
  params:
    UTC_timestamp -  UTC timestamp in milliseconds, integer
  returns:
    string - datetime in string format '%Y-%m-%d %H:%M:%S'
  ''' 
  UTC_timestamp = int(UTC_timestamp/1000.0)
  return datetime.datetime.fromtimestamp(UTC_timestamp)

def convert_UTC_timestamp_to_date(UTC_timestamp):
  '''
  Converts UTC timestamp in milliseconds, integer, into string date fromated '%Y-%m-%d %H:%M:%S'
  params:
    UTC_timestamp -  UTC timestamp in milliseconds, integer
  returns:
    string - datetime in string format '%Y-%m-%d %H:%M:%S'
  ''' 
  UTC_timestamp = int(UTC_timestamp/1000.0)
  return datetime.datetime.fromtimestamp(UTC_timestamp).strftime('%Y-%m-%d %H:%M:%S')




def get_OHLCV(exchange, symbol, timeframe, since=None, to_date=None, limit=None, verbose=1):
  '''
  https://docs.ccxt.com/en/latest/manual.html#ohlcv-candlestick-charts

  params:
    exchange - CCXT exchange object
    symbol - ex: "BTC/USDT"
    timeframe - '1m', '5m', '15m', '1h', '1d', '1w','1M' other if supported by exchange
    since - date from when to fetch data with format YYYYmmDD or YYYYmmDDHHMMSS  ex: 20170101 or 20170101000001
    to_date - date to when to fetch data with format YYYYmmDD or YYYYmmDDHHMMSS  ex: 20170101 or 20170101000001
    limit - number of candles to fetch

  returns: 
    list - lits (a flat array) of OHLCV candles 
            [
              [
                  1504541580000, // UTC timestamp in milliseconds, integer
                  4235.4,        // (O)pen price, float
                  4240.6,        // (H)ighest price, float
                  4230.0,        // (L)owest price, float
                  4230.7,        // (C)losing price, float
                  37.72941911    // (V)olume (in terms of the base currency), float
              ],
              ...
            ]
  '''
  
  ohlcv = []

  if since:
    since = convert_date_to_UTC_timestamp(since)
  
  if to_date:
    to_date = convert_date_to_UTC_timestamp(to_date)
  else:
    to_date =  int(time.mktime(datetime.datetime.now().timetuple()) * 1000)
  
    
  if exchange.has['fetchOHLCV']:
      ## fetchOHLCV (symbol, timeframe = '1m', since = undefined, limit = undefined, params = {})
      ## The since argument is an integer UTC timestamp in milliseconds (everywhere throughout the library with all unified methods).
      if since: 
        proceed = True
        while proceed:
          fetched_ohlcv = exchange.fetch_ohlcv(symbol, since=since, limit= limit, timeframe=timeframe)
          
          if len(fetched_ohlcv):
            since = fetched_ohlcv[-1][0] + 1 # gets the last timestamp and add 1
            if since > to_date:
              fetched_ohlcv = [x for x in fetched_ohlcv if x[0] <= to_date ]
              proceed = False            
            
            ohlcv += fetched_ohlcv

            
            first_date = convert_UTC_timestamp_to_date(ohlcv[0][0])
            last_date = convert_UTC_timestamp_to_date(ohlcv[-1][0])
            
            if verbose == 1:
              print("Fetched:", len(ohlcv), "| First date:", first_date, "| Last date:", last_date)

          else:
            break
      else:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)



  return np.array(ohlcv) 


def get_DF_OHLCV(exchange, symbol="BTC/USDT", timeframe="1m", since="201708170400", to_date="20220101", limit=None, save_dir="/content/drive/MyDrive/ColabData/OHLCV/BTC", verbose=1):
  '''
  params:
    exchange - CCXT exchange object
    symbol - ex: "BTC/USDT"
    timeframe - '1m', '5m', '15m', '1h', '1d', '1w','1M' other if supported by exchange
    since - date from when to fetch data with format YYYYmmDD or YYYYmmDDHHMMSS  ex: 20170101 or 20170101000001
    to_date - date to when to fetch data with format YYYYmmDD or YYYYmmDDHHMMSS  ex: 20170101 or 20170101000001
    limit - number of candles to fetch

  returns:
    Dataframe - columns[ 
                        UTC_timestamp, // UTC timestamp in milliseconds, integer
                        Open,        // (O)pen price, float
                        High,        // (H)ighest price, float
                        Low,        // (L)owest price, float
                        Close,        // (C)losing price, float
                        Volume    // (V)olume (in terms of the base currency), float
                ]
  '''
  
  
  if not os.path.exists(save_dir):
    os.mkdir(save_dir)

  file_path = f"{save_dir}/{symbol.replace('/','_')}_{timeframe}_{since}_{to_date}.csv"

  if(os.path.exists(file_path)):
    if(verbose==1):
      print("reading csv file: ", file_path)
    df = pd.read_csv(file_path, index_col = 0)
    
    df.index= pd.DatetimeIndex(df.index, dtype='datetime64[ns]', freq=None)

  else:
    ohlcv = get_OHLCV(exchange, symbol,timeframe, since, to_date, verbose=verbose) 
    columns = ["UTC_timestamp","Open", "High", "Low", "Close", "Volume" ]
    df = pd.DataFrame(ohlcv,columns=columns) 
    datetime_index = df.UTC_timestamp.apply(convert_UTC_timestamp_to_date_obj)
    df.index= pd.DatetimeIndex(datetime_index, dtype='datetime64[ns]', freq=None)

    # df.set_index('datetime', inplace=True)
    df.drop(columns=["UTC_timestamp"], inplace=True)
    df.to_csv(file_path)
  

  df.info()


    


  

  return df

